In [16]:
import ee

In [17]:
ee.Initialize()

In [18]:
# define globals
AOI = ee.FeatureCollection("users/ryangilberthamilton/demo/region_124").geometry()
START, END = "2020-06-01", "2020-09-01"

In [19]:
# helpers for pre processing SAR and Optical Collections
def maskS2clouds(image: ee.Image) -> ee.Image:
    qa = image.select("QA60")
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask)

def ndvi(image: ee.Image) -> ee.Image:
    """ computes NDVI for a given Sentinel 2 image  """
    return image.normalizedDifference(["B8", "B4"]).rename("NDVI")

def denoise(image: ee.Image) -> ee.Image:
    """ applies a 3x3 boxcar to the image
    r: 1 = 3x3, 2 = 5x5, 3 = 7x7, etc.
    radius sets the mid point of the kernel
    """
    return image.convolve(ee.Kernel.square(1))

def ratio(image):
    """ computes the ratio of VV/VH for a given Sentinel 1 image """
    return image.select("VV").divide(image.select("VH")).rename("VV/VH")

# Collection Preprocessing
- apply meta filters
- select bands we want to keep
- apply pre processing functions
- reduce collection to single image

In [20]:
s1_dv: ee.Image = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filterBounds(AOI)
    .filterDate(START, END)
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .map(denoise)
    .map(lambda image: image.addBands(ratio(image)))
    .median()
    .select(['V.*'])
    .clip(AOI)
)

s2_sr: ee.Image = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') 
    .filterBounds(AOI) 
    .filterDate(START, END) 
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(maskS2clouds)
    .map(lambda image: image.addBands(ndvi(image)))
    .median()
    .select('B.*|NDVI') # select all spectral bands and NDVI
    .clip(AOI)
)

dem = ee.Image("NASA/NASADEM_HGT/001").select("elevation").rename("DEM")
# compute slope from dem
slope = ee.Terrain.slope(dem).rename("slope")
dem = dem.addBands(slope)

# create the stack
# stack ee.Image.cat(*args)
stack = ee.Image.cat(s1_dv, s2_sr, dem)

In [21]:
from pprint import pprint
pprint(stack.bandNames().getInfo())

['VV',
 'VH',
 'VV/VH',
 'B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'NDVI',
 'DEM',
 'slope']


In [22]:
# visualize 
import geemap 

Map = geemap.Map()

Map.addLayer(s1_dv, {'min': -25, 'max': 0, 'bands': ['VV']}, 'S1: VV') 
Map.addLayer(s2_sr, {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B4']}, 'S2: (B4, B3, B2)')

Map.centerObject(AOI, 10)
Map.addLayerControl()
Map

Map(center=[44.379560465104134, -65.07312485382938], controls=(WidgetControl(options=['position', 'transparent…

# Build Features
- apply a map to replace class names with numbers
- merge both train and test collections (import for doing assessment)
- assign type 1, 2 for train and test
- sample the stack to get features for Smile Random Forest Classifier

In [23]:
# build features for training
train = ee.FeatureCollection("users/ryangilberthamilton/demo/ns_124_train") 
test = ee.FeatureCollection("users/ryangilberthamilton/demo/ns_124_test")

# apply data type 
train = train.map(lambda feature: feature.set("type", 1))
test = test.map(lambda feature: feature.set("type", 2))

features = ee.FeatureCollection([train, test]).flatten()

pprint(features.first().getInfo())

{'geometry': {'coordinates': [-65.18775035580974, 44.60486999736613],
              'type': 'Point'},
 'id': '0_00000000000000000000',
 'properties': {'class_name': 'Marsh', 'type': 1},
 'type': 'Feature'}


In [24]:
# remap the str type to int
LABEL_COL = 'class_name'

old_labels: ee.List = features.aggregate_array(LABEL_COL).distinct().sort()
new_labels: ee.List = ee.List.sequence(1, old_labels.size())
print(f"Old labels: {old_labels.getInfo()}")
print(f"New labels: {new_labels.getInfo()}")

Old labels: ['Bog', 'Fen', 'Marsh', 'Open water', 'Salt marsh', 'Shallow water', 'Swamp', 'Upland']
New labels: [1, 2, 3, 4, 5, 6, 7, 8]


In [25]:
# remap the labels to int rperesentation
features = features.remap(old_labels, new_labels, LABEL_COL)
pprint(features.first().getInfo())

{'geometry': {'coordinates': [-65.18775035580974, 44.60486999736613],
              'type': 'Point'},
 'id': '0_00000000000000000000',
 'properties': {'class_name': 3, 'type': 1},
 'type': 'Feature'}


In [26]:
sample = stack.sampleRegions(
    collection=features, 
    properties=[LABEL_COL, 'type'], 
    tileScale=16,
    scale=10, 
    geometries=False
)

In [27]:
pprint(sample.first().getInfo()) # THIS MAY TAKE A WHILE DEPENDING ON THE SIZE OF THE FEATURE COLLECTION OR STACK

{'geometry': None,
 'id': '0_00000000000000000000_0',
 'properties': {'B1': 289.5,
                'B11': 1088.5,
                'B12': 565.5,
                'B2': 313,
                'B3': 605.5,
                'B4': 399.5,
                'B5': 1261.5,
                'B6': 2500.5,
                'B7': 2936.5,
                'B8': 3079,
                'B8A': 3335.5,
                'B9': 3270.5,
                'DEM': 183,
                'NDVI': 0.7499844431877136,
                'VH': -17.44650696146597,
                'VV': -9.691074788409026,
                'VV/VH': 0.5669369848521929,
                'class_name': 3,
                'slope': 1.3024958372116089,
                'type': 1},
 'type': 'Feature'}


# Random Forest Classifier
- train classifier
- display classified image
- apply classifier to test data
- assess accuracy for the test data


In [28]:
# model
class SmileRandomForest:
    def __init__(self, **kwargs):
        """ a simple wrapper for the ee.Classifier.smileRandomForest """
        self.model = ee.Classifier.smileRandomForest(**kwargs)

    def train(self, sample: ee.FeatureCollection, label_col: str):
        self.model = self.model.train(
            features=sample, 
            classProperty=label_col, 
            inputProperties=stack.bandNames()
        )
        return self

    def predict(self, X: ee.Image | ee.FeatureCollection) -> ee.Image:
        return X.classify(self.model)

model = SmileRandomForest(numberOfTrees=100, maxNodes=1000)
pprint(model.model.getInfo())



{'maxNodes': 1000, 'numberOfTrees': 100, 'type': 'Classifier.smileRandomForest'}


In [29]:
# trian the model
train_features = sample.filter('type == 1')

model.train(train_features, LABEL_COL)
predict = model.predict(stack)

In [30]:
# visualize
CMap = geemap.Map()
CMap.addLayer(predict, {'min': 1, 'max': 8}, 'Predicted') # use gray scale for the predicted classes, simpler to visualize
CMap.addLayerControl()
CMap.centerObject(AOI, 10)
CMap


Map(center=[44.379560465104134, -65.07312485382938], controls=(WidgetControl(options=['position', 'transparent…

In [31]:
# accuracy assessment
test_features = sample.filter('type == 2')
test_predict = model.predict(test_features)

cfm = test_predict.errorMatrix(LABEL_COL, 'classification', order=new_labels)
print(f"Confusion Matrix: ")
cfm


Confusion Matrix: 
